This notebook takes the enumerated jobs in the previous notebook and makes ure

In [1]:
import geopandas as gpd
import yaml
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import asf_search as asf
from datetime import timedelta
from dateparser import parse
import concurrent.futures
import hyp3_sdk
import numpy as np
from enum_utils import get_enum_dir, get_stack_dir

# Setup

In [2]:
YAML_FILE = 'enumeration_parameters.yml'
with open(YAML_FILE) as f:
    enum_params = yaml.safe_load(f)['enumeration_parameters']

In [3]:
AOI_NAME = enum_params['aoi_name']
NEIGHBORS = enum_params['neighbors']

In [4]:
stack_dir = get_stack_dir(AOI_NAME)
enum_dir = get_enum_dir(AOI_NAME, NEIGHBORS)
assert enum_dir.exists()

In [5]:
AOI_NAME, enum_dir, stack_dir

('us_east',
 PosixPath('out/us_east/enum/neighbors_3'),
 PosixPath('out/us_east/stack'))

# Read enumeration data

In [6]:
dfs = [gpd.read_file(p) for p in tqdm(list(enum_dir.glob('*.geojson')))]
df_ifg = pd.concat(dfs, axis=0)
df_ifg.head()

100%|███████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.76it/s]


,reference,secondary,reference_date,secondary_date,frame_id,track_aoi_key,aoi_name,neighbors,geometry
0,S1A_IW_SLC__1SDV_20230721T225841_20230721T2259...,S1A_IW_SLC__1SDV_20230709T225840_20230709T2259...,2023-07-21,2023-07-09,16417.0,track106,us_east,3.0,"POLYGON Z ((-76.48738 37.52033 0.00000, -76.69..."
1,S1A_IW_SLC__1SDV_20230721T225907_20230721T2259...,S1A_IW_SLC__1SDV_20230709T225906_20230709T2259...,2023-07-21,2023-07-09,16418.0,track106,us_east,3.0,"POLYGON Z ((-76.76865 38.68045 0.00000, -77.02..."
2,S1A_IW_SLC__1SDV_20230721T225907_20230721T2259...,S1A_IW_SLC__1SDV_20230709T225906_20230709T2259...,2023-07-21,2023-07-09,16419.0,track106,us_east,3.0,"POLYGON Z ((-77.05876 39.85632 0.00000, -77.31..."
3,S1A_IW_SLC__1SDV_20230721T225932_20230721T2259...,S1A_IW_SLC__1SDV_20230709T225931_20230709T2259...,2023-07-21,2023-07-09,16420.0,track106,us_east,3.0,"POLYGON Z ((-77.35151 41.00204 0.00000, -77.39..."
4,S1A_IW_SLC__1SDV_20230721T225956_20230721T2300...,S1A_IW_SLC__1SDV_20230709T225956_20230709T2300...,2023-07-21,2023-07-09,16421.0,track106,us_east,3.0,"POLYGON Z ((-77.64872 42.16093 0.00000, -77.78..."


# Filtering Jobs (Optional)

When first submitting jobs, you may just want to submit one track at a time. Easiest to filter on the dataframe with all the IFGs. That's what we will do. Below shows how to filter on a single "track_aoi_id".

In [7]:
# df_ifg_submission = df_ifg.copy()

In [8]:
df_ifg_submission = df_ifg[df_ifg.track_aoi_key == 'track4'].reset_index(drop=True)
df_ifg_submission.head()

,reference,secondary,reference_date,secondary_date,frame_id,track_aoi_key,aoi_name,neighbors,geometry
0,S1A_IW_SLC__1SDV_20230807T230554_20230807T2306...,S1A_IW_SLC__1SDV_20230714T230553_20230714T2306...,2023-08-07,2023-07-14,492.0,track4,us_east,3.0,"POLYGON Z ((-77.82534 34.47332 0.00000, -78.06..."
1,S1A_IW_SLC__1SDV_20230807T230619_20230807T2306...,S1A_IW_SLC__1SDV_20230714T230618_20230714T2306...,2023-08-07,2023-07-14,493.0,track4,us_east,3.0,"POLYGON Z ((-78.09591 35.63539 0.00000, -78.33..."
2,S1A_IW_SLC__1SDV_20230807T230644_20230807T2307...,S1A_IW_SLC__1SDV_20230714T230643_20230714T2307...,2023-08-07,2023-07-14,494.0,track4,us_east,3.0,"POLYGON Z ((-78.37797 36.81111 0.00000, -78.41..."
3,S1A_IW_SLC__1SDV_20230807T230709_20230807T2307...,S1A_IW_SLC__1SDV_20230714T230708_20230714T2307...,2023-08-07,2023-07-14,495.0,track4,us_east,3.0,"POLYGON Z ((-78.65322 37.95742 0.00000, -78.73..."
4,S1A_IW_SLC__1SDV_20230807T230709_20230807T2307...,S1A_IW_SLC__1SDV_20230714T230708_20230714T2307...,2023-08-07,2023-07-14,496.0,track4,us_east,3.0,"POLYGON Z ((-78.93684 39.11929 0.00000, -79.10..."


In [9]:
df_ifg_submission.shape

(9416, 9)

# Deduplication of IFGs CMR - WIP - Requires CMR Ingest

Once we have CMR ingesting jobs, we can use this as the reference for data has been processed. We will use the fixed frames. See this [PR](https://github.com/asfadmin/Discovery-asf_search/issues/198) for more details about how this feature is access through ASF Search.

The raw CMR command for a comparable query is:
```
https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=SENTINEL-1_INTERFEROGRAMS&temporal=2022-02-24T00:00:00Z,2022-02-25T00:00:00Z&attribute[]=int,FRAME_NUMBER,25502&attribute[]=int,TEMPORAL_BASELINE_DAYS,12
```

In [10]:
def get_gunw_hits(record):

    frame_id = int(record['frame_id'])
    ref_date = parse(record['reference_date'])
    sec_date = parse(record['secondary_date'])
    
    start = ref_date - timedelta(days=.5)
    end = ref_date + timedelta(days=.5)
    tb_days = (ref_date - sec_date).days
    

    opts = asf.ASFSearchOptions(host='cmr.uat.earthdata.nasa.gov', 
                                platform=asf.SENTINEL1,
                                processingLevel=asf.constants.GUNW_STD,
                                frame=frame_id, 
                                start=start, 
                                end=end, 
                                temporalBaselineDays=[tb_days],
                                maxResults=5
                                )
    
    results = asf.search(opts=opts)
    return len(results)

In [11]:
submission_records = df_ifg_submission.to_dict('records')
#get_gunw_hits(submission_records[0])

In [12]:
# n = len(submission_records)
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     ifg_hits_cmr = list(tqdm(executor.map(get_gunw_hits, submission_records[:100]), total=n))

In [13]:
submission_records[0]

{'reference': 'S1A_IW_SLC__1SDV_20230807T230554_20230807T230621_049776_05FC51_A90F S1A_IW_SLC__1SDV_20230807T230619_20230807T230646_049776_05FC51_59B8',
 'secondary': 'S1A_IW_SLC__1SDV_20230714T230553_20230714T230620_049426_05F188_7C0A S1A_IW_SLC__1SDV_20230714T230618_20230714T230645_049426_05F188_7317',
 'reference_date': '2023-08-07',
 'secondary_date': '2023-07-14',
 'frame_id': 492.0,
 'track_aoi_key': 'track4',
 'aoi_name': 'us_east',
 'neighbors': 3.0,
 'geometry': <POLYGON Z ((-77.825 34.473 0, -78.064 35.483 0, -78.06 35.484 0, -78.061 35...>}

In [14]:
# TODO: set to ifg_hits_cmr
df_ifg_submission['cmr_hits'] = 0

# Format Job Parameters

We format the dictionary of inputs for the interferograms (IFGs) into the format required by Hyp3 (just reformats into another dictionary).

In [15]:
IONO_CORRECTION = True
SET_CORRECTION = True
weather_model = 'HRRR'

In [16]:
submission_records[0]

{'reference': 'S1A_IW_SLC__1SDV_20230807T230554_20230807T230621_049776_05FC51_A90F S1A_IW_SLC__1SDV_20230807T230619_20230807T230646_049776_05FC51_59B8',
 'secondary': 'S1A_IW_SLC__1SDV_20230714T230553_20230714T230620_049426_05F188_7C0A S1A_IW_SLC__1SDV_20230714T230618_20230714T230645_049426_05F188_7317',
 'reference_date': '2023-08-07',
 'secondary_date': '2023-07-14',
 'frame_id': 492.0,
 'track_aoi_key': 'track4',
 'aoi_name': 'us_east',
 'neighbors': 3.0,
 'geometry': <POLYGON Z ((-77.825 34.473 0, -78.064 35.483 0, -78.06 35.484 0, -78.061 35...>}

In [17]:
def generate_job_name(record):
    track_key = record['track_aoi_key']
    aoi_name = record['aoi_name']
    neighbor = str(int(record['neighbors']))
    # frame_id = str(int(record['frame_id']))
    return f'{track_key}-n{neighbor}'

In [18]:
job_names = list(map(generate_job_name, submission_records))
job_names_unique = sorted(list(set(job_names)))
job_names_unique[:5]

['track4-n3']

In [19]:
job_parameters = [{'granules': record['reference'].split(' '),
                   'secondary_granules': record['secondary'].split(' '),
                   'frame_id': int(record['frame_id']),
                   'weather_model': weather_model,
                   'estimate_ionosphere_delay': IONO_CORRECTION,
                   'compute_solid_earth_tide': SET_CORRECTION,
                  } for record in submission_records]

job_dicts = [{'name': job_name,
              # NOTE: we are still using the `dev` branch. Change this to "INSAR_ISCE" to use the `main` branch.
              'job_type': 'INSAR_ISCE_TEST',
              'job_parameters': parameters
             }
             for parameters, job_name in zip(job_parameters, job_names) ]
job_dicts[:2]

[{'name': 'track4-n3',
  'job_type': 'INSAR_ISCE_TEST',
  'job_parameters': {'granules': ['S1A_IW_SLC__1SDV_20230807T230554_20230807T230621_049776_05FC51_A90F',
    'S1A_IW_SLC__1SDV_20230807T230619_20230807T230646_049776_05FC51_59B8'],
   'secondary_granules': ['S1A_IW_SLC__1SDV_20230714T230553_20230714T230620_049426_05F188_7C0A',
    'S1A_IW_SLC__1SDV_20230714T230618_20230714T230645_049426_05F188_7317'],
   'frame_id': 492,
   'weather_model': 'HRRR',
   'estimate_ionosphere_delay': True,
   'compute_solid_earth_tide': True}},
 {'name': 'track4-n3',
  'job_type': 'INSAR_ISCE_TEST',
  'job_parameters': {'granules': ['S1A_IW_SLC__1SDV_20230807T230619_20230807T230646_049776_05FC51_59B8',
    'S1A_IW_SLC__1SDV_20230807T230644_20230807T230711_049776_05FC51_52AC'],
   'secondary_granules': ['S1A_IW_SLC__1SDV_20230714T230618_20230714T230645_049426_05F188_7317',
    'S1A_IW_SLC__1SDV_20230714T230643_20230714T230710_049426_05F188_8808'],
   'frame_id': 493,
   'weather_model': 'HRRR',
   'est

# Submit Jobs to Hyp3

This is where we take the job dictionaries and submit them to Hyp3. Note, we have different AWS accounts associated to different API endpoints i.e. `hyp3-a19-jpl` and `hyp3-tibet-jpl` are different AWS accounts using the same workflow and are separated for accounting purposes.

In [20]:
# uses .netrc; add `prompt=True` to prompt for credentials; 
#hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu')
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-tibet-jpl.asf.alaska.edu')

Hyp3 requires us to submit 200 jobs at a time.

In [21]:
N_batches = int(np.ceil(len(job_dicts) / 200))
job_dicts_batches = [job_dicts[200 * k : 200 * (k + 1)] for k in range(N_batches)]

Below these are what should be uncommented to actually submit the jobs.

In [22]:
# # For testing
# submitted_jobs = list(hyp3_isce.submit_prepared_jobs(job_dicts_batches[0][:]))

In [23]:
# submitted_jobs = []
# for job_dict in tqdm(job_dicts_batches[1:]):
#     submitted_jobs += list(hyp3_isce.submit_prepared_jobs(job_dict[:]))

Check status

In [24]:
for job_name in job_names_unique[:5]:
    jobs = hyp3_isce.find_jobs(name=job_name)
    print('#' * 10)
    print(job_name)
    print(jobs)

##########
track4-n3
9416 HyP3 Jobs: 9181 succeeded, 235 failed, 0 running, 0 pending.


# Failed jobs

In [25]:
job_names_unique[0]

'track4-n3'

In [26]:
failed_jobs = hyp3_isce.find_jobs(name=job_names_unique[0], status_code='FAILED')
dicts_failed = [j.to_dict() for j in failed_jobs]
dicts_failed[-3:]

[{'job_id': '38934b6c-9770-469f-b3fd-64852193fc75',
  'job_type': 'INSAR_ISCE_TEST',
  'request_time': '2023-08-17T20:30:41+00:00',
  'status_code': 'FAILED',
  'user_id': 'cmarshak',
  'name': 'track4-n3',
  'job_parameters': {'compute_solid_earth_tide': True,
   'estimate_ionosphere_delay': True,
   'frame_id': 496,
   'granules': ['S1A_IW_SLC__1SDV_20230807T230709_20230807T230736_049776_05FC51_0329',
    'S1A_IW_SLC__1SDV_20230807T230734_20230807T230801_049776_05FC51_7667'],
   'secondary_granules': ['S1A_IW_SLC__1SDV_20230608T230705_20230608T230732_048901_05E173_82F6',
    'S1A_IW_SLC__1SDV_20230608T230730_20230608T230757_048901_05E173_21C5'],
   'weather_model': 'HRRR'},
  'logs': ['https://hyp3-tibet-jpl-contentbucket-81rn23hp7ppf.s3.us-west-2.amazonaws.com/38934b6c-9770-469f-b3fd-64852193fc75/38934b6c-9770-469f-b3fd-64852193fc75.log'],
  'expiration_time': '2023-09-02T00:00:00+00:00',
  'processing_times': [9362.393, 6.399]},
 {'job_id': '7d1974bc-b857-4f54-9c9b-5c8756bf4a3d',
 